---
## Helpful Comments & Instructions
---

In [1]:
# This file is a example/template for the `read_data.py` script, that loads the
# data from a specific paper and reformats it into the standard format used by
# SPAG. To use this, copy the following function into a testing script or
# Jupyter notebook, and modify the function name and contents as needed.


# Specifically, change all instances of...

# - `authorYYYYx` to the first author's last name (maybe with their first 
# initial of there is a duplicate last name) and year of the paper. When there
# are instances of an author releasing more than one paper in a year, append a
# letter to the year in `x` position.
#   (e.g., `Ji et al. 2016a` --> `ji2016a`, if there is also a 
#       `Ji et al. 2016b`)
#   (e.g., `Hansen et al. 2018` --> `hansent2018c`, if there are multiple
#       authors with the same last name and this was T. Hansen's 3rd paper in 
#       2018)

# - the path to the data files to point to the correct location of the data 
#   tables.
#   (e.g., `data_dir + "abundance_tables/authorYYYYx/table1.csv"`)

# - The `Reference` and `Ref` columns to match the citation for the paper.
#   --> `Reference` column should be in the form 'Author+Year'.
#       (e.g., 'Ji+2016a', 'HansenT+2018c')
#   --> `Ref` column should shortened form of the reference, typically the 
#       first 3 letters of the author's last name (all uppercase) and the last
#       two digits of the year, with a letter appended if there are multiple
#       papers by the same author(s) in that year. (e.g.'JI16a', 'HANt18c')

# - The `Loc` column to match the type of object the stars are in. Use:
#   --> 'HA' for halo stars
#   --> 'BU' for bulge stars
#   --> 'DS' for disk stars
#   --> 'DW' for dwarf galaxy stars
#   --> 'UF' for ultra-faint dwarf galaxy stars
#   --> 'GC' for globular cluster stars

# - The `System` column to match the name of the system the stars are in, if 
#   applicable and not already in the data table.


# Beyond this, confirm that the datafiles are reading in correctly, and that
# the columns are being filled in properly. You may need to modify how the data 
# is read in or the structure of the datafile themselves, depending on the 
# format of the data tables.

# Things to be wary of...

# - Some papers use different formats for the species names, such as "Fe I" vs
#   "FeI" vs "Fe1". The `ion_to_species()` function should be able to handle 
#   most of these, but you may need to modify the datafile if there are any 
#   issues, or contact A. Yelland.

# - Some papers may not provide all the necessary columns, such as separate
#   upper/lower limit flag column. You may need to modify the datafile to add
#   the flag columns.

# - Additionally, different authors at different times may use different solar
#   abundance scales. The `get_solar()` function can be used to retrieve the
#   solar abundances of the most commonly used scales, but if one is not
#   available, please let A. Yelland know.
  

---
## Package Imports & Directories
---

In [3]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from __future__ import (division, print_function, absolute_import,
                        unicode_literals)
import  sys, os, glob, time, IPython

import astropy.constants as const
import astropy.units as u
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
# from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, EarthLocation

# from PyAstronomy import pyasl

import matplotlib as mpl
import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec

import numpy as np
import scipy as sp
import pandas as pd

import seaborn as sns
sns.set_palette("colorblind")
colors = sns.color_palette("colorblind", 20)

# from smh import Session

from spag.read_data import *
from spag.convert import *
from spag.utils import *
# from spag.calculate import *
# import spag.read_data as rd
import spag.coordinates as scoord

# import alexmods.read_data as rd

script_dir = "/".join(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[:-1]) + "/"
# script_dir = os.path.dirname(os.path.realpath(__file__))+"/"
data_dir = '/Users/ayelland/Research/metal-poor-stars/spag/spag/data_templates/'
plotting_dir = script_dir+"plots/"
if not os.path.exists(plotting_dir):
    os.makedirs(plotting_dir)


---
## Template Function
---

In [5]:
def load_authorYYYYx(io=None):
    """
    Load the Author et al. YYYYx data for the <grouping or classification of stars>.

    Table 1 - Observations
    Table 2 - Stellar Parameters
    Table 3 - Abundance Table
    """

    ## Read in the data tables
    obs_df = pd.read_csv(data_dir + "authorYYYYx/table1.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])
    param_df = pd.read_csv(data_dir + "authorYYYYx/table3.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])
    abund_df = pd.read_csv(data_dir + "authorYYYYx/table4.csv", comment="#", na_values=["", " ", "nan", "NaN", "N/A", "n/a"])

    ## Make the new column names
    species = []
    for ion in abund_df["Species"].unique():
        species_i = ion_to_species(ion)
        elem_i = ion_to_element(ion)
        if species_i not in species:
            species.append(species_i)

    epscols = [make_epscol(s) for s in species]
    ulcols = [make_ulcol(s) for s in species]
    XHcols = [make_XHcol(s).replace(" ", "") for s in species]
    ulXHcols = ['ul' + col for col in XHcols]
    XFecols = [make_XFecol(s).replace(" ", "") for s in species]
    ulXFecols = ['ul' + col for col in XFecols]
    errcols = [make_errcol(s) for s in species]

    ## New dataframe with proper columns
    authorYYYYx_df = pd.DataFrame(
                    columns=['I/O','Name','Simbad_Identifier','Reference','Ref','Loc','System','RA_hms','RA_deg','DEC_dms','DEC_deg',
                    'Teff','logg','Fe/H','Vmic'] + epscols + ulcols + XHcols + ulXHcols + XFecols 
                    + ulXFecols + errcols)
    for i, name in enumerate(abund_df['Name'].unique()):
        authorYYYYx_df.loc[i,'Name'] = name
        authorYYYYx_df.loc[i,'Simbad_Identifier'] = obs_df.loc[obs_df['Name'] == name, 'Simbad_Identifier'].values[0]
        authorYYYYx_df.loc[i,'Reference'] = 'Author+YYYYx'
        authorYYYYx_df.loc[i,'Ref'] = 'AUTYYx'
        authorYYYYx_df.loc[i,'I/O'] = 1
        authorYYYYx_df.loc[i,'Loc'] = 'UF' # [HA, BU, DS, DW, UF, GC]
        authorYYYYx_df.loc[i,'System'] = obs_df.loc[obs_df['Name'] == name, 'System'].values[0]
        authorYYYYx_df.loc[i,'RA_hms'] = obs_df.loc[obs_df['Name'] == name, 'RA_hms'].values[0]
        authorYYYYx_df.loc[i,'RA_deg'] = scoord.ra_hms_to_deg(authorYYYYx_df.loc[i,'RA_hms'], precision=6)
        authorYYYYx_df.loc[i,'DEC_dms'] = obs_df.loc[obs_df['Name'] == name, 'DEC_dms'].values[0]
        authorYYYYx_df.loc[i,'DEC_deg'] = scoord.dec_dms_to_deg(authorYYYYx_df.loc[i,'DEC_dms'], precision=2)
        authorYYYYx_df.loc[i,'Teff'] = param_df.loc[param_df['Name'] == name, 'Teff'].values[0]
        authorYYYYx_df.loc[i,'logg'] = param_df.loc[param_df['Name'] == name, 'logg'].values[0]
        authorYYYYx_df.loc[i,'Fe/H'] = param_df.loc[param_df['Name'] == name, 'Fe/H'].values[0]
        authorYYYYx_df.loc[i,'Vmic'] = param_df.loc[param_df['Name'] == name, 'Vmic'].values[0]

        ## Fill in data
        star_df = abund_df[abund_df['Name'] == name]
        for j, row in star_df.iterrows():
            ion = row["Species"]
            species_i = ion_to_species(ion)
            elem_i = ion_to_element(ion)

            logepsX_sun_a09 = get_solar(elem_i, version='asplund2009')[0]
            logepsFe_a09 = star_df.loc[star_df['Species'] == 'Fe I', 'logepsX'].values[0]
            feh_a09 = logepsFe_a09 - get_solar('Fe', version='asplund2009')[0]

            ## Assign epsX values
            col = make_epscol(species_i)
            if col in epscols:
                authorYYYYx_df.loc[i, col] = row["logepsX"] if pd.isna(row["l_logepsX"]) else np.nan

            ## Assign ulX values
            col = make_ulcol(species_i)
            if col in ulcols:
                authorYYYYx_df.loc[i, col] = row["logepsX"] if pd.notna(row["l_logepsX"]) else np.nan

            ## Assign [X/H] and ul[X/H]values
            col = make_XHcol(species_i).replace(" ", "")
            if col in XHcols:
                if pd.isna(row["l_[X/H]"]):
                    authorYYYYx_df.loc[i, col] = normal_round(row["logepsX"] - logepsX_sun_a09, 2)
                    authorYYYYx_df.loc[i, 'ul'+col] = np.nan
                else:
                    authorYYYYx_df.loc[i, col] = np.nan
                    authorYYYYx_df.loc[i, 'ul'+col] = normal_round(row["logepsX"] - logepsX_sun_a09, 2)
                if 'e_[X/H]' in row.index:
                    authorYYYYx_df.loc[i, 'e_'+col] = row["e_[X/H]"]

            ## Assign [X/Fe] values
            col = make_XFecol(species_i).replace(" ", "")
            if col in XFecols:
                if pd.isna(row["l_[X/Fe]"]):
                    authorYYYYx_df.loc[i, col] = normal_round((row["logepsX"] - logepsX_sun_a09) - feh_a09, 2)
                    authorYYYYx_df.loc[i, 'ul'+col] = np.nan
                else:
                    authorYYYYx_df.loc[i, col] = np.nan
                    authorYYYYx_df.loc[i, 'ul'+col] = normal_round((row["logepsX"] - logepsX_sun_a09) - feh_a09, 2)
                if 'e_[X/Fe]' in row.index:
                    authorYYYYx_df.loc[i, 'e_'+col] = row["e_[X/Fe]"]

            ## Assign error values
            col = make_errcol(species_i)
            if col in errcols:
                e_logepsX = row.get("e_logepsX", np.nan)
                if pd.notna(e_logepsX):
                    authorYYYYx_df.loc[i, col] = e_logepsX
                else:
                    authorYYYYx_df.loc[i, col] = np.nan

    ## Drop the Fe/Fe columns
    authorYYYYx_df.drop(columns=['[Fe/Fe]','ul[Fe/Fe]','[FeII/Fe]','ul[FeII/Fe]'], inplace=True, errors='ignore')

    return authorYYYYx_df

df = load_authorYYYYx()
display(df)

KeyError: 'Simbad_Identifier'

---
## Testing Zone
---

In [ ]:
# This is where you can copy/paste the above function, and modify it for your specific reference.

---
## Helpful Conversion Scripts for Different Datafiles & Numbers
---

### Abundance Calculations

In [3]:
## Quick Abundance Conversions

### Solar Abundances
epsfe_sun = rd.get_solar('Fe')[0]
epsba_sun = rd.get_solar('Ba')[0]
epssr_sun = rd.get_solar('Sr')[0]

### From Anna's notes of a presentation
srfe = -1.15
epsba = -3.5
feh = -5.62

### Calculated log(eps) values
srh = srfe + feh
epssr = epssr_sun + srh
epsfe = epsfe_sun + feh

feh_new = epsfe - epsfe_sun
srh_new = epssr - epssr_sun
bah_new = epsba - epsba_sun

srfe_new = srh_new - feh_new
bafe_new = bah_new - feh_new

print(f"logeps(Fe):  {epsfe:.2f}")
print(f"logeps(Sr): <{epssr:.2f}")
print(f"logeps(Ba):  {epsba:.2f}")
print(f"[Fe/H]:      {feh_new:.2f}")
print(f"[Sr/H]:     <{srh_new:.2f}")
print(f"[Ba/H]:      {bah_new:.2f}")
print(f"[Sr/Fe]:    <{srfe_new:.2f}")
print(f"[Ba/Fe]:     {bafe_new:.2f}")

logeps(Fe):  1.88
logeps(Sr): <-3.90
logeps(Ba):  -3.50
[Fe/H]:      -5.62
[Sr/H]:     <-6.77
[Ba/H]:      -5.68
[Sr/Fe]:    <-1.15
[Ba/Fe]:     -0.06


In [4]:
XH_from_eps(0.73, 'Sr')

-2.14

### CDS $\rightarrow$ CSV

In [ ]:
## Convert all CDS Standard Formatted Tables to CSVs for Stellar Abundances
create_csv_files = True
if create_csv_files:
    system =  'cayrel2004'
    table_numbers = [2,3,8]
    for n in table_numbers:

        base_path = f"/Users/ayelland/Research/metal-poor-stars/spag/spag/data_templates/cayrel2004/"
        table_path = base_path + f"table{n}.dat"
        readme_path = base_path + f"ReadMe"
        csv_path = base_path + f"table{n}.csv"

        if os.path.exists(table_path) and os.path.exists(readme_path):
            table_data = ascii.read(table_path, format='cds', readme=readme_path)
                    # guess=False,
                    # fast_reader=False
                    
            table_data = table_data.to_pandas()
            table_data.to_csv(csv_path, index=False)
        else:
            print(f"Table {n} or ReadMe file does not exist for {system}.")


In [ ]:
## Convert CDS Standard Formatted Tables to CSVs for Galaxy Properties

from astropy.io import ascii
import os

create_csv_files = True
if create_csv_files:
    system = 'mcconnachie2012'
    table_numbers = [1, 2, 3, 4, 5]

    base_path = f"/Users/ayelland/Research/metal-poor-stars/spag/spag/data/galaxy_properties/{system}/"
    readme_path = os.path.join(base_path, "ReadMe")

    for n in table_numbers:
        table_path = os.path.join(base_path, f"table{n}.dat")
        csv_path = os.path.join(base_path, f"table{n}.csv")

        if os.path.exists(table_path) and os.path.exists(readme_path):
            try:
                table_data = ascii.read(
                    table_path,
                    format='cds',
                    readme=readme_path,
                    guess=False,
                    fast_reader=False,
                    fill_values=[('-', 'nan'), ('--', 'nan'), ('...', 'nan')]
                )
                table_data = table_data.to_pandas()
                table_data.to_csv(csv_path, index=False)
                print(f"✅ Converted table{n} to CSV.")
            except Exception as e:
                print(f"❌ Failed to read table{n}: {e}")
        else:
            print(f"⚠️  Missing table{n}.dat or ReadMe file for {system}.")


✅ Converted table1 to CSV.
❌ Failed to read table2: Column V(MW) failed to convert: invalid literal for int() with base 10: 'nan'
✅ Converted table3 to CSV.
✅ Converted table4 to CSV.
✅ Converted table5 to CSV.


### HMS/DMS $\leftrightarrow$ Degrees

In [ ]:
## Convert coordinates between degrees and hms/dms format

data = '''
242.63458901020093, -8.260693273003724 
226.25888112673744, -13.738953843436517
272.43169186461097, -40.86888561990257 
'''

def convert_deg_to_hmsdms(data):
    """
    Convert the given data string of coordinates from degrees to hms/dms format.
    """
    coords = []
    for line in data.strip().split('\n'):
        ra_deg, dec_deg = map(float, line.split(','))
        ra_hms = scoord.ra_deg_to_hms(ra_deg, precision=2)
        dec_dms = scoord.dec_deg_to_dms(dec_deg, precision=2)
        coords.append((ra_hms, dec_dms))
        # print(f"{ra_hms}, {dec_dms}")
    return coords

def convert_hmsdms_to_deg(data):
    """
    Convert the given data string of coordinates from degrees to hms/dms format.
    """
    coords = []
    for line in data.strip().split('\n'):
        ra_hms, dec_hms = map(str, line.split(','))
        ra_deg = scoord.ra_hms_to_deg(ra_hms, precision=6)
        dec_deg = scoord.dec_dms_to_deg(dec_hms, precision=6)
        coords.append((ra_deg, dec_deg))
        # print(f"{ra_deg}, {dec_deg}")

    return coords

# Convert the coordinates
try:
    txt = convert_deg_to_hmsdms(data)
except:
    try:
        txt = convert_hmsdms_to_deg(data)
    except Exception as e:
        print(f"Error converting coordinates: {e}")
        sys.exit(1)

print("Converted Coordinates:\n")
for c in txt:
    print(f'({c[0]} , {c[1]})')

Converted Coordinates:

(16:10:32.30 , -08:15:38.50)
(15:05:02.13 , -13:44:20.23)
(18:09:43.61 , -40:52:07.99)
